<a href="https://colab.research.google.com/github/emherraiz/Trabajo_Final/blob/main/Trabajo_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import dask.dataframe as dd
import pandas as pd
from jinja2 import Template

##  Cargaremos el conjunto de datos en un dataframe.

In [9]:
# Especifica la ruta del archivo CSV
archivo_csv = "air_traffic_data.csv"

# Lee el archivo CSV en un dataframe de Dask
datos_traf = dd.read_csv(archivo_csv)

In [10]:
# Vamos a obtener informaión sobre la estructura de los datos:
campos = datos_traf.columns
tipos = datos_traf.dtypes

# Cargar la plantilla desde un archivo o una cadena
with open('plantilla_datos.html') as f:
    plantilla = Template(f.read())


In [11]:
# Renderizar la plantilla con los datos
datos = zip(campos, tipos)
informe = plantilla.render(datos=datos)

# Guardar el informe en un archivo como se nos pide en el primer ejercicio
with open('informe.html', 'w') as f:
    f.write(informe)

# Leer el archivo HTML y extraer las tablas
tablas = pd.read_html('informe.html')


## ¿Con qué tipo de datos contamos?

In [12]:
# Obtener la primera tabla (si hay varias tablas en el archivo HTML)
tabla = tablas[0]

# Imprimir el DataFrame de la tabla
print(tabla)
print('\n\nLos datos se han cargado correctamente, hemos guardado el informe en el archivo informe.html\n\n')

# Análisis de los datos
num_filas = len(datos_traf)  # Número de filas en el DataFrame
num_columnas = len(datos_traf.columns)  # Número de columnas en el DataFrame
columnas = datos_traf.columns.tolist()  # Lista de

               Nombre del campo Tipo de dato
0               Activity Period        int64
1             Operating Airline       object
2   Operating Airline IATA Code       object
3             Published Airline       object
4   Published Airline IATA Code       object
5                   GEO Summary       object
6                    GEO Region       object
7            Activity Type Code       object
8           Price Category Code       object
9                      Terminal       object
10                Boarding Area       object
11              Passenger Count        int64
12  Adjusted Activity Type Code       object
13     Adjusted Passenger Count        int64
14                         Year        int64
15                        Month       object


Los datos se han cargado correctamente, hemos guardado el informe en el archivo informe.html




##  ¿Cuántas compañías diferentes aparecen en el fichero?

In [13]:
# Información sobre las columnas
info_columnas = []
for columna in columnas:
    tipo_dato = datos_traf[columna].dtype  # Tipo de dato de la columna
    valores_unicos = datos_traf[columna].nunique().compute()  # Número de valores únicos en la columna
    info_columnas.append({'Nombre del campo': columna, 'Tipo de dato': tipo_dato, 'Valores únicos': valores_unicos})

# Presentar los resultados
for columna_info in info_columnas:
    print(f"Nombre del campo: {columna_info['Nombre del campo']}")
    print(f"Tipo de dato: {columna_info['Tipo de dato']}")
    print(f"Número de valores únicos: {columna_info['Valores únicos']}")
    print()

Nombre del campo: Activity Period
Tipo de dato: int64
Número de valores únicos: 129

Nombre del campo: Operating Airline
Tipo de dato: object
Número de valores únicos: 77

Nombre del campo: Operating Airline IATA Code
Tipo de dato: object
Número de valores únicos: 73

Nombre del campo: Published Airline
Tipo de dato: object
Número de valores únicos: 68

Nombre del campo: Published Airline IATA Code
Tipo de dato: object
Número de valores únicos: 64

Nombre del campo: GEO Summary
Tipo de dato: object
Número de valores únicos: 2

Nombre del campo: GEO Region
Tipo de dato: object
Número de valores únicos: 9

Nombre del campo: Activity Type Code
Tipo de dato: object
Número de valores únicos: 3

Nombre del campo: Price Category Code
Tipo de dato: object
Número de valores únicos: 2

Nombre del campo: Terminal
Tipo de dato: object
Número de valores únicos: 5

Nombre del campo: Boarding Area
Tipo de dato: object
Número de valores únicos: 8

Nombre del campo: Passenger Count
Tipo de dato: int64


In [14]:
# Obtener las compañías únicas de la columna "Operating Airline"
companias_operating = datos_traf['Operating Airline'].nunique().compute()
operadora = datos_traf['Operating Airline']

# Obtener las compañías únicas de la columna "Published Airline"
companias_published = datos_traf['Published Airline'].nunique().compute()
publicada = datos_traf['Published Airline']

# Imprimir los resultados
print(f"Número de compañías en la columna 'Operating Airline': {companias_operating}")
print(f"Número de compañías en la columna 'Published Airline': {companias_published}")


Número de compañías en la columna 'Operating Airline': 77
Número de compañías en la columna 'Published Airline': 68


In [21]:
# Realizar un merge basado en las columnas "Operating Airline" y "Published Airline"
merged_data = dd.merge(datos_traf[['Operating Airline']], datos_traf[['Published Airline']], left_index=True, right_index=True)

# Filtrar los datos donde las columnas "Operating Airline" y "Published Airline" no coinciden
mismatched_data = merged_data[merged_data['Operating Airline'] != merged_data['Published Airline']].compute()

# Imprimir los datos que no coinciden
print("\nDatos que no coinciden entre 'Operating Airline' y 'Published Airline':\n")
print(mismatched_data)
print('\nCon un total de', len(mismatched_data), 'datos que no coinciden')



Datos que no coinciden entre 'Operating Airline' y 'Published Airline':

                 Operating Airline  Published Airline
25         American Eagle Airlines  American Airlines
26         American Eagle Airlines  American Airlines
29     Atlantic Southeast Airlines    Delta Air Lines
30     Atlantic Southeast Airlines    Delta Air Lines
47                    Horizon Air     Alaska Airlines
...                            ...                ...
14958             SkyWest Airlines    United Airlines
14959             SkyWest Airlines    United Airlines
14960             SkyWest Airlines    United Airlines
14961             SkyWest Airlines    United Airlines
14962             SkyWest Airlines    United Airlines

[1602 rows x 2 columns]

Con un total de 1602 datos que no coinciden


In [16]:
# Calcular el número medio de pasajeros por compañía en la columna "Operating Airline"
pasajeros_por_compania = datos_traf.groupby('Operating Airline')['Passenger Count'].mean().compute()

# Imprimir el número medio de pasajeros por compañía
print("Número medio de pasajeros por compañía:")
print(pasajeros_por_compania)

Número medio de pasajeros por compañía:
Operating Airline
ATA Airlines          8744.636364
Aer Lingus            4407.183673
Aeromexico            5463.822222
Air Berlin            2320.750000
Air Canada           18251.560109
                         ...     
Virgin Atlantic       9847.104651
WestJet Airlines      5338.155340
World Airways          261.666667
XL Airways France     2223.161290
Xtra Airways            73.000000
Name: Passenger Count, Length: 77, dtype: float64


In [18]:
# Ordenar el DataFrame por número de pasajeros en orden descendente dentro de cada grupo "GEO Región"
df_sorted = datos_traf.sort_values('Passenger Count', ascending=False)

# Eliminar los registros duplicados por "GEO Región" y mantener solo aquellos con el mayor número de pasajeros
df_unique = df_sorted.drop_duplicates(subset='GEO Region', keep='first').compute()

# Imprimir el DataFrame resultante
print("Registros únicos por 'GEO Región' con mayor número de pasajeros:")
print(df_unique)

Registros únicos por 'GEO Región' con mayor número de pasajeros:
       Activity Period                 Operating Airline  \
11331           201308                   United Airlines   
2973            200708  United Airlines - Pre 07/01/2013   
14008           201507                   United Airlines   
2872            200708                       Air Canada    
12590           201407                   United Airlines   
13945           201507                         Emirates    
13191           201501                   Air New Zealand   
12903           201410                              TACA   
7737            201101                          LAN Peru   

      Operating Airline IATA Code                 Published Airline  \
11331                          UA                   United Airlines   
2973                           UA  United Airlines - Pre 07/01/2013   
14008                          UA                   United Airlines   
2872                           AC                 

In [19]:
# Guardar los resultados en un archivo CSV
pasajeros_por_compania.to_csv('ruta_del_archivo_pasajeros_por_compania.csv', header=True, index=False)
df_unique.to_csv('ruta_del_archivo_regiones_unicas.csv', header=True, index=False)

In [20]:
# Calcular la media de cada columna
media = datos_traf.mean().compute()

# Calcular la desviación estándar de cada columna
desviacion_estandar = datos_traf.std().compute()

# Imprimir los resultados obtenidos
print("Media de cada elemento del conjunto de datos:")
print(media)

print("\nDesviación estándar de cada elemento del conjunto de datos:")
print(desviacion_estandar)

Media de cada elemento del conjunto de datos:
Activity Period             201045.073366
Passenger Count              29240.521090
Adjusted Passenger Count     29331.917105
Year                          2010.385220
dtype: float64

Desviación estándar de cada elemento del conjunto de datos:
Activity Period               313.336196
Passenger Count             58319.509284
Adjusted Passenger Count    58284.182219
Year                            3.137589
dtype: float64
